# Summer School: Digitale Methoden der Zeitungsanalyse

*Hinweis:* Dieses Notebook enthält nur den Python-Code. Über „Run -> Run All Cells“ kann die Zeitung „[La otra Alemania](https://www.deutsche-digitale-bibliothek.de/newspaper/2149754-0)“ heruntergeladen werden.

In [ ]:
# benötigt Python-Bibliotheken installieren
%pip install -q pysolr pandas requests lxml

## Suchindex `newspaper`

In [ ]:
import pysolr

# Solr-Endpunkt-URL
solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper'

# Solr-Client initialisieren
solr = pysolr.Solr(solr_url, timeout=10)

# Suchparameter
q = {
    'q': 'location:"buenos aires" AND hasLoadedIssues:true',
    'fl': 'id,title,location,frequency,progress',
    'rows': 10  # Anzahl der zurückzugebenden Ergebnisse
}

# Suche ausführen
results = solr.search(**q)

# Ergebnisse ausgeben
for result in results:
    print(f"ID: {result.get('id', 'N/A')}")
    print(f"Title: {result.get('title', 'N/A')}")
    print(f"Location: {result.get('location', 'N/A')}")
    print(f"Frequency: {result.get('frequency', 'N/A')}")
    print(f"Progress: {result.get('progress', 'N/A')}")
    print("-" * 40)

## Suchindex `newspaper-issues`

In [ ]:
import pandas as pd

solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues'
solr = pysolr.Solr(solr_url, always_commit=True, timeout=10)

q = {
    'q': 'zdb_id:2149754-0 AND type:issue',
    'rows': 1000
}

response = solr.search(**q)
    
# Überführen der Ergebnisse in ein Pandas DataFrame
df = pd.DataFrame(response.docs)

# DataFrame anzeigen
df

### Erste Datenanalyse

In [ ]:
# Sicherstellen, dass publication_date als Datumswerte formatiert sind
df['publication_date'] = pd.to_datetime(df['publication_date'])

# Frühestes und spätestes Datum ermitteln
earliest_date = df['publication_date'].min()
latest_date = df['publication_date'].max()

# Ergebnisse anzeigen
print(f"Frühestes Veröffentlichungsdatum: {earliest_date}")
print(f"Spätestes Veröffentlichungsdatum: {latest_date}")

## Download der METS/MODS-Daten

In [ ]:
import os
import requests

# Verzeichnis für die XML-Dateien erstellen
directory = 'La_Otra_Alemania'
if not os.path.exists(directory):
    os.makedirs(directory)

# Durch jede Zeile des DataFrames iterieren
for index, row in df.iterrows():
    # Extrahiere den Wert der Spalte id
    item_id = row['id']
    
    # Formatiere den Wert der Spalte publication_date im Format YYYY-MM-DD
    publication_date = row['publication_date'].strftime('%Y-%m-%d')
    
    # Generiere die URL zur API-Abfrage
    url = f'https://api.deutsche-digitale-bibliothek.de/2/items/{item_id}/source/record'
    
    # Setze die HTTP-Header
    headers = {
        'Accept': 'application/xml'
    }
    
    # API-Anfrage senden
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Erstelle einen Dateipfad für die XML-Datei
        file_path = os.path.join(directory, f'{publication_date}_{item_id}.xml')
        
        # Speichere die XML-Datei im erstellten Verzeichnis
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'Datei gespeichert: {file_path}')
    else:
        print(f'Fehler beim Abrufen der Datei für ID {item_id}: {response.status_code}')

print('Fertig!')

## Download der Bild- und Volltextdaten

In [ ]:
from lxml import etree

# Definiere die Namensräume
NAMESPACES = {
    'mets': 'http://www.loc.gov/METS/',
    'xlink': 'http://www.w3.org/1999/xlink'
}

# Verzeichnisse definieren
xml_directory = 'La_Otra_Alemania'
download_directory = 'La_Otra_Alemania'

# Erstelle das Download-Verzeichnis, falls es nicht existiert
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Funktion, um URLs aus einer XML-Datei zu extrahieren und herunterzuladen
def download_files_from_xml(xml_file_path, xpath_expr, subfolder, extension):
    # Lade die XML-Datei ein
    with open(xml_file_path, 'rb') as xml_file:
        tree = etree.parse(xml_file)

    # Extrahiere die URLs mit dem gegebenen XPath-Ausdruck
    urls = tree.xpath(xpath_expr, namespaces=NAMESPACES)

    # Erstelle das Unterverzeichnis, benannt nach der XML-Datei
    xml_file_name = os.path.splitext(os.path.basename(xml_file_path))[0]
    destination_dir = os.path.join(download_directory, xml_file_name, subfolder)
    os.makedirs(destination_dir, exist_ok=True)

    # Lade jede URL herunter und speichere die Datei
    for i, url in enumerate(urls, start=1):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Überprüfe auf HTTP-Fehler
            file_path = os.path.join(destination_dir, f'{i}.{extension}')
            with open(file_path, 'wb') as output_file:
                output_file.write(response.content)
            print(f'Downloaded {url} to {file_path}')
        except requests.RequestException as e:
            print(f'Failed to download {url}: {e}')

# Durchlaufe alle XML-Dateien im Verzeichnis
for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_file_path = os.path.join(xml_directory, xml_file)
        # Lade und speichere Dateien mit dem ersten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href', 'DEFAULT', 'jpeg')
        # Lade und speichere Dateien mit dem zweiten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href', 'DDB_FULLTEXT', 'xml')
